<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">Modèles pour la Prédiction des Pannes (Modèle 1)</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
    <li><strong>Réseaux de neurones récurrents (RNN) :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Notamment <strong>LSTM</strong> (Long Short-Term Memory) et <strong>GRU</strong> (Gated Recurrent Units), pour capturer les dépendances temporelles.</li>
        </ul>
    </li>
    <li><strong>Temporal Fusion Transformer (TFT) :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Un modèle avancé adapté aux séries temporelles multivariées, capable de gérer les variables continues et catégoriques tout en capturant les relations complexes.</li>
        </ul>
    </li>
    <li><strong>Gradient Boosting (XGBoost, LightGBM, CatBoost) :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Pour exploiter les relations non linéaires et les interactions entre les variables.</li>
        </ul>
    </li>
    <li><strong>Forêts aléatoires (Random Forest) :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Pour une approche robuste et interprétable de la classification.</li>
        </ul>
    </li>
    <li><strong>Support Vector Machines (SVM) :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Utile pour des prédictions précises dans des contextes bien définis, mais nécessite un prétraitement rigoureux.</li>
        </ul>
    </li>
    <li><strong>Autoencoders :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Pour détecter les anomalies en apprenant la représentation normale des données.</li>
        </ul>
    </li>
</ul>
</ul>


<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">Observations sur les Approches Modélisées</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
    <li><strong>Observation Unique :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Chaque ligne du dataset (une observation avec ses features) est traitée séparément.</li>
            <li>Le modèle n'a pas de notion de dépendance temporelle entre les observations.</li>
        </ul>
    </li>
    <li><strong>Fenêtre Temporelle (non utilisée dans Random Forest classique) :</strong>
        <ul style="font-size: 11pt; color: #555;">
            <li>Les fenêtres temporelles sont couramment utilisées dans des modèles spécifiques aux séries temporelles, comme :</li>
            <ul style="font-size: 11pt; color: #555;">
                <li><strong>LSTM</strong> / <strong>GRU</strong> (réseaux récurrents).</li>
                <li><strong>TFT</strong> (Temporal Fusion Transformer).</li>
                <li><strong>ARIMA, SARIMA, Prophet</strong>, etc.</li>
            </ul>
            <li>Dans ce cas, les observations environnantes sont prises en compte pour capturer les relations temporelles.</li>
        </ul>
    </li>
</ul>


In [1]:
import pandas as pd
import numpy as np

In [12]:
# Charger les données
df = pd.read_csv("../Datasources/MetroPT3_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

In [ ]:
display(df)

<h3 style="text-align: center; font-family: Arial, sans-serif; color: RED;">################# DÉCLARATION / INITIALISATION #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [13]:
# Convertir timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
#display(df.dtypes)

In [14]:
continuous_features = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers", "LPS", "Pressure_switch", "Oil_level", "Caudal_impulses"]

In [15]:
# Conserver uniquement les colonnes continues, catégorielles et 'timestamp'
columns_to_keep = ["timestamp", "panne"] + continuous_features + categorical_features
df = df[columns_to_keep]

In [7]:
display(df)

,timestamp,panne,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,COMP,DV_eletric,Towers,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,0,-0.012,-0.024,53.600,0.0400,9.358,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,0,-0.014,-0.022,53.675,0.0400,9.348,1.0,0.0,1.0,0.0,1.0,1.0,1.0
2,2020-02-01 00:00:20,0,-0.012,-0.022,53.600,0.0425,9.338,1.0,0.0,1.0,0.0,1.0,1.0,1.0
3,2020-02-01 00:00:30,0,-0.012,-0.022,53.425,0.0400,9.328,1.0,0.0,1.0,0.0,1.0,1.0,1.0
4,2020-02-01 00:00:40,0,-0.012,-0.022,53.475,0.0400,9.318,1.0,0.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841755,2020-09-01 03:59:10,0,-0.014,-0.022,59.675,0.0425,8.918,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1841756,2020-09-01 03:59:20,0,-0.014,-0.020,59.600,0.0450,8.904,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1841757,2020-09-01 03:59:30,0,-0.014,-0.022,59.600,0.0425,8.892,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1841758,2020-09-01 03:59:40,0,-0.012,-0.022,59.550,0.0450,8.878,1.0,0.0,1.0,0.0,1.0,1.0,1.0


<h3 style="text-align: center; font-family: Arial, sans-serif; color: RED;">#################### FIN DÉCLARATION #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
# Filtrer les colonnes sans "_is_missing"
columns_without_is_missing = [col for col in df.columns if not col.endswith('_is_missing')]

In [ ]:
# Colonnes marquant les valeurs manquantes (is_missing)
cols_is_missing = [col + '_is_missing' for col in ['TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs', 
                                                   'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric', 
                                                   'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level', 
                                                   'Caudal_impulses']]

In [ ]:
# Convertir les colonnes catégoriques en type category
for col in categorical_features:
    df[col] = df[col].astype("category")


In [ ]:
# Statistiques descriptives globales (exclure les colonnes `_is_missing`)
pd.options.display.float_format = '{:,.2f}'.format

# Statistiques descriptives globales
print("Statistiques descriptives :")
display(df[columns_without_is_missing].describe())

# Statistiques pour les colonnes spécifiques (exclure les colonnes `_is_missing`)
# Statistiques pour les colonnes spécifiques
print("\nNombre de valeurs manquantes par colonne :")
missing_counts = df.isnull().sum()
missing_percent = (df.isnull().mean() * 100).map("{:,.2f}%".format)
missing_stats = pd.DataFrame({
    'Valeurs manquantes': missing_counts.map("{:,}".format),
    'Pourcentage manquant': missing_percent
})

# Afficher les statistiques
display(missing_stats)


In [ ]:
pd.set_option('display.max_columns', None)
display(df.head(5))

<ul style="font-family: times, serif; font-size:14pt; color:blue;">
<strong>MODELE - Random Forest</strong>
</ul>

<ul style="font-family: times, serif; font-size:14pt; color:RED;">
<strong>TEST 1 : sans fenetre de prediction mais par observation unique</strong>
</ul>

In [42]:
###################################################################
################ Train : Panne1 & Panne2 & Panne3  ################
################ Test  : Panne4           #########################
###################################################################

# Définir les limites pour le train et le test
train_start = pd.Timestamp('2020-02-01 00:00:00')
train_end   = pd.Timestamp('2020-06-08 14:30:00')
test_start  = pd.Timestamp('2020-06-08 14:30:10')  # Observation suivante
test_end    = pd.Timestamp('2020-09-01 03:59:50')

# Filtrer les indices pour le train et le test
train_indices = df[(df['timestamp'] >= train_start) & (df['timestamp'] <= train_end)].index
test_indices  = df[(df['timestamp'] >= test_start) & (df['timestamp'] <= test_end)].index

# Créer les ensembles d'entraînement et de test
X_train = df.loc[train_indices, continuous_features + categorical_features]
y_train = df.loc[train_indices, 'panne']

X_test = df.loc[test_indices, continuous_features + categorical_features]
y_test = df.loc[test_indices, 'panne']

# Vérifications des tailles des ensembles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")
print(f"Pannes dans l'entraînement : {y_train.sum()} ({y_train.mean():.2%})")
print(f"Pannes dans le test : {y_test.sum()} ({y_test.mean():.2%})")


Taille de l'ensemble d'entraînement : 1111141
Taille de l'ensemble de test : 730619
Pannes dans l'entraînement : 30957 (2.79%)
Pannes dans le test : 1981 (0.27%)


In [41]:
# Entraîner le modèle Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entraînement du modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)

Rapport de classification :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    728818
           1       0.06      0.00      0.00      1621
           2       0.00      0.00      0.00       180

    accuracy                           1.00    730619
   macro avg       0.35      0.33      0.33    730619
weighted avg       1.00      1.00      1.00    730619

Importance des features :
            Feature  Importance
1       DV_pressure    0.397753
2   Oil_temperature    0.264562
4        Reservoirs    0.150677
0               TP2    0.106710
3     Motor_current    0.060686
5              COMP    0.005960
6        DV_eletric    0.004122
11  Caudal_impulses    0.003123
7            Towers    0.002399
10        Oil_level    0.002352
8               LPS    0.001180
9   Pressure_switch    0.000475


<ul style="font-family: times, serif; font-size:14pt; color:RED;">
<strong>TEST 2 : sans fenetre de prediction mais par observation unique</strong>
</ul>

In [43]:
###################################################################
################ Train : Panne1 & Panne2  ################
################ Test  : Panne3           #########################
###################################################################


# Définir les périodes pour le train et le test
train_periods = [{'start': '2020-02-01 00:00:00', 'end': '2020-05-30 06:00:00'}]

test_periods  = [{'start': '2020-05-30 06:00:10', 'end': '2020-06-07 14:30:00'}]

# Filtrer les indices pour le train
train_indices = []
for period in train_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    train_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Filtrer les indices pour le test
test_indices = []
for period in test_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    test_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Vérifier qu'il n'y a pas de chevauchement entre train et test
assert len(set(train_indices).intersection(set(test_indices))) == 0, "Les ensembles d'entraînement et de test se chevauchent !"

# Créer les ensembles d'entraînement et de test
X_train = df.loc[train_indices, continuous_features + categorical_features]
y_train = df.loc[train_indices, 'panne']

X_test = df.loc[test_indices, continuous_features + categorical_features]
y_test = df.loc[test_indices, 'panne']

# Vérifications des tailles des ensembles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")
print(f"Pannes dans l'entraînement : {y_train.sum()} ({y_train.mean():.2%})")
print(f"Pannes dans le test : {y_test.sum()} ({y_test.mean():.2%})")


Taille de l'ensemble d'entraînement : 1030321
Taille de l'ensemble de test : 72180
Pannes dans l'entraînement : 11696 (1.14%)
Pannes dans le test : 19261 (26.68%)


In [44]:
# Entraîner le modèle Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entraînement du modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)

Rapport de classification :
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     53099
           1       0.99      0.33      0.49     18901
           2       0.00      0.00      0.00       180

    accuracy                           0.82     72180
   macro avg       0.60      0.44      0.46     72180
weighted avg       0.85      0.82      0.78     72180

Importance des features :
            Feature  Importance
1       DV_pressure    0.337021
4        Reservoirs    0.298922
2   Oil_temperature    0.151718
0               TP2    0.141438
3     Motor_current    0.057051
5              COMP    0.004082
11  Caudal_impulses    0.004000
6        DV_eletric    0.002823
7            Towers    0.001325
10        Oil_level    0.001229
9   Pressure_switch    0.000255
8               LPS    0.000137


<ul style="font-family: times, serif; font-size:14pt; color:RED;">
<strong>TEST 3 : sans fenetre de prediction mais par observation unique</strong>
</ul>

In [ ]:
###################################################################
################ Train : Panne2 & Panne3 & Panne4  ################
################ Test  : Panne1           #########################
###################################################################


# Définir les périodes pour le train et le test
train_periods = [{'start': '2020-04-18 23:59:10', 'end': '2020-09-01 03:59:50'}]

test_periods  = [{'start': '2020-02-01 00:00:00', 'end': '2020-04-18 23:59:00'}]

# Filtrer les indices pour le train
train_indices = []
for period in train_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    train_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Filtrer les indices pour le test
test_indices = []
for period in test_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    test_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Vérifier qu'il n'y a pas de chevauchement entre train et test
assert len(set(train_indices).intersection(set(test_indices))) == 0, "Les ensembles d'entraînement et de test se chevauchent !"

# Créer les ensembles d'entraînement et de test
X_train = df.loc[train_indices, continuous_features + categorical_features]
y_train = df.loc[train_indices, 'panne']

X_test = df.loc[test_indices, continuous_features + categorical_features]
y_test = df.loc[test_indices, 'panne']

# Vérifications des tailles des ensembles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")
print(f"Pannes dans l'entraînement : {y_train.sum()} ({y_train.mean():.2%})")
print(f"Pannes dans le test : {y_test.sum()} ({y_test.mean():.2%})")


In [47]:
# Entraîner le modèle Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entraînement du modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)

Rapport de classification :
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    665100
           1       0.05      0.02      0.03      8635
           2       0.00      0.00      0.00       180

    accuracy                           0.98    673915
   macro avg       0.35      0.34      0.34    673915
weighted avg       0.97      0.98      0.98    673915

Importance des features :
            Feature  Importance
2   Oil_temperature    0.449541
1       DV_pressure    0.275703
4        Reservoirs    0.124612
3     Motor_current    0.063869
0               TP2    0.062224
6        DV_eletric    0.008788
5              COMP    0.006130
8               LPS    0.003335
10        Oil_level    0.002498
7            Towers    0.001611
9   Pressure_switch    0.001148
11  Caudal_impulses    0.000542


<ul style="font-family: times, serif; font-size:14pt; color:RED;">
<strong>TEST 4 : sans fenetre de prediction mais par observation unique</strong>
</ul>

In [49]:
###################################################################
################ Train : Panne2 & Panne3          ################
################ Test  : Panne4           #########################
###################################################################
# Définir les périodes pour le train et le test
train_periods = [{'start': '2020-04-18 23:59:10', 'end': '2020-07-10 14:29:50'}]

test_periods  = [{'start': '2020-07-10 14:30:00', 'end': '2020-09-01 03:59:50'}]

# Filtrer les indices pour le train
train_indices = []
for period in train_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    train_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Filtrer les indices pour le test
test_indices = []
for period in test_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    test_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Vérifier qu'il n'y a pas de chevauchement entre train et test
assert len(set(train_indices).intersection(set(test_indices))) == 0, "Les ensembles d'entraînement et de test se chevauchent !"

# Créer les ensembles d'entraînement et de test
X_train = df.loc[train_indices, continuous_features + categorical_features]
y_train = df.loc[train_indices, 'panne']

X_test = df.loc[test_indices, continuous_features + categorical_features]
y_test = df.loc[test_indices, 'panne']

# Vérifications des tailles des ensembles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")
print(f"Pannes dans l'entraînement : {y_train.sum()} ({y_train.mean():.2%})")
print(f"Pannes dans le test : {y_test.sum()} ({y_test.mean():.2%})")

Taille de l'ensemble d'entraînement : 713705
Taille de l'ensemble de test : 454140
Pannes dans l'entraînement : 21962 (3.08%)
Pannes dans le test : 1981 (0.44%)


In [50]:
# Entraîner le modèle Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entraînement du modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)

Rapport de classification :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    452339
           1       0.00      0.00      0.00      1621
           2       0.00      0.00      0.00       180

    accuracy                           1.00    454140
   macro avg       0.33      0.33      0.33    454140
weighted avg       0.99      1.00      0.99    454140

Importance des features :
            Feature  Importance
2   Oil_temperature    0.465423
1       DV_pressure    0.277837
4        Reservoirs    0.109867
0               TP2    0.073029
3     Motor_current    0.053944
5              COMP    0.006074
6        DV_eletric    0.005192
10        Oil_level    0.002947
8               LPS    0.002274
11  Caudal_impulses    0.001638
7            Towers    0.001565
9   Pressure_switch    0.000210


<ul style="font-family: times, serif; font-size:14pt; color:RED;">
<strong>TEST 5 : sans fenetre de prediction mais par observation unique</strong>
</ul>

In [7]:
###################################################################
################ Train : Panne1 & Panne2          ################
################ Test  : Panne3           #########################
###################################################################
# Définir les périodes pour le train et le test
train_periods = [{'start': '2020-02-01 00:00:00', 'end': '2020-06-04 09:59:50'}]

test_periods  = [{'start': '2020-06-04 10:00:00', 'end': '2020-07-14 14:29:50'}]

# Filtrer les indices pour le train
train_indices = []
for period in train_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    train_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Filtrer les indices pour le test
test_indices = []
for period in test_periods:
    start = pd.Timestamp(period['start'])
    end = pd.Timestamp(period['end'])
    test_indices.extend(df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].index.tolist())

# Vérifier qu'il n'y a pas de chevauchement entre train et test
assert len(set(train_indices).intersection(set(test_indices))) == 0, "Les ensembles d'entraînement et de test se chevauchent !"

# Créer les ensembles d'entraînement et de test
X_train = df.loc[train_indices, continuous_features + categorical_features]
y_train = df.loc[train_indices, 'panne']

X_test = df.loc[test_indices, continuous_features + categorical_features]
y_test = df.loc[test_indices, 'panne']

# Vérifications des tailles des ensembles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")
print(f"Pannes dans l'entraînement : {y_train.sum()} ({y_train.mean():.2%})")
print(f"Pannes dans le test : {y_test.sum()} ({y_test.mean():.2%})")

Taille de l'ensemble d'entraînement : 1074960
Taille de l'ensemble de test : 347220
Pannes dans l'entraînement : 11696 (1.09%)
Pannes dans le test : 19261 (5.55%)


In [53]:
# Entraîner le modèle Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entraînement du modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)

Rapport de classification :
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    328139
           1       0.99      0.20      0.34     18901
           2       0.00      0.00      0.00       180

    accuracy                           0.96    347220
   macro avg       0.65      0.40      0.44    347220
weighted avg       0.96      0.96      0.94    347220

Importance des features :
            Feature  Importance
1       DV_pressure    0.340700
4        Reservoirs    0.305047
2   Oil_temperature    0.148733
0               TP2    0.132631
3     Motor_current    0.057027
5              COMP    0.006078
11  Caudal_impulses    0.003527
6        DV_eletric    0.002979
7            Towers    0.001349
10        Oil_level    0.001184
9   Pressure_switch    0.000664
8               LPS    0.000081


<ul style="font-family: times, serif; font-size:14pt; color:RED;">
<strong>TEST 6 : FENETRES DE 15 MIN<strong>
</ul>

In [16]:
#######################################################
###########  REDEFINIR LA VARIABLE 'PANNE' ###########
### PANNE = 0 : PAS DE PANNE               ###########
### PANNE = 1 : EN PLEIN PANNE             ###########
### PANNE = 2 : 15 MINUTES AVANT LA PANNE  ###########
######################################################
## CORRECTION ##


warning_duration=15                    # Les minutes ou 'PANNE' = 2
    
## Traitement du dataset en adequation avec le window de 15 mn

# Mise à jour de la colonne 'panne' pour refléter les nouveaux critères
def update_panne_column(df, periodes_pannes, warning_duration):
    df['panne'] = 0  # Initialiser toutes les valeurs à 0

    for periode in periodes_pannes:
        start_panne = pd.Timestamp(periode['start'])
        end_panne = pd.Timestamp(periode['end'])
        warning_start = start_panne - pd.Timedelta(minutes=warning_duration)

        # Mettre à jour les valeurs de panne : 1 pour les périodes de panne
        df.loc[(df['timestamp'] >= start_panne) & (df['timestamp'] <= end_panne), 'panne'] = 1
        
        # Mettre à jour les valeurs d'avertissement : 2 pour les 15 minutes avant la panne
        df.loc[(df['timestamp'] >= warning_start) & (df['timestamp'] < start_panne), 'panne'] = 2

    return df

# Appliquer la mise à jour
periodes_pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
]

df_updated = update_panne_column(df, periodes_pannes, warning_duration)

# Vérification des résultats
print("Terminé")
print(" 4 pannes X 15(mn) * 6 (Nbr de gap par minute)")
print("--------------------------------------")
print("Window =  15 mn  ==> 360 Observations")
print("Window =  30 mn  ==> 720 Observations")
print("Window =  60 mn  ==> 1440 Observations")
print("--------------------------------------")
# Vérifier les modalités et leurs occurrences dans la colonne 'panne'
if 'panne' in df.columns:
    panne_modalities = df['panne'].value_counts()
    print("Modalités de la colonne 'panne' avec leurs occurrences :")
    print(panne_modalities)
else:
    print("La colonne 'panne' n'existe pas dans le DataFrame.")


Terminé
 4 pannes X 15(mn) * 6 (Nbr de gap par minute)
--------------------------------------
Window =  15 mn  ==> 360 Observations
Window =  30 mn  ==> 720 Observations
Window =  60 mn  ==> 1440 Observations
--------------------------------------
Modalités de la colonne 'panne' avec leurs occurrences :
0    1809902
1      31498
2        360
Name: panne, dtype: int64


In [17]:
# DEFINIR LE FENETRAGE

## window_size =  15 mn  ==> 90   Observations
## window_size =  30 mn  ==> 180  Observations
## window_size =  60 mn  ==> 360  Observations

window_size = 90 

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings

# Masquer les warnings spécifiques aux performances de DataFrame fragmenté
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)


# Créer une fenêtre temporelle
def create_temporal_window(df_entree, features, window_size):
    """
    Ajoute des lags pour créer une fenêtre temporelle pour les features spécifiées.

    Args:
    - df : DataFrame original.
    - features : Liste des colonnes pour lesquelles créer les lags.
    - window_size : Taille de la fenêtre temporelle (nombre d'observations précédentes).
    
    Returns:
    - df_windowed : DataFrame avec les lags ajoutés.
    """
    df_windowed = df_entree.copy()
    for lag in range(1, window_size):
        for feature in features:
            df_windowed[f'{feature}_lag_{lag}'] = df[feature].shift(lag)
    
    # Supprimer les lignes affectées par les décalages
    df_windowed = df_windowed.iloc[window_size - 1:].reset_index(drop=True)
    return df_windowed

# Taille de la fenêtre (90 observations)
window_size = 90
continuous_features = ['TP2', 'DV_pressure', 'Oil_temperature', 'Motor_current', 'Reservoirs']

# Appliquer la création de fenêtres temporelles
df_windowed = create_temporal_window(df_updated, continuous_features, window_size)

# Vérification des dimensions
print(f"Taille originale : {df.shape}")
print(f"Taille avec fenêtres temporelles : {df_windowed.shape}")


Taille originale : (1841760, 14)
Taille avec fenêtres temporelles : (1841671, 459)


In [18]:
# Enregistrer le dataframe corrigé dans un fichier CSV
df_windowed.to_csv("../Datasources/MetroPT3_corrected_windowed.csv", index=True)

In [ ]:
# Charger les données
df_windowed = pd.read_csv("../Datasources/MetroPT3_corrected_windowed.csv", delimiter=",", decimal=".", index_col=0)
df_windowed.reset_index(drop=True, inplace=True)

# Convertir timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
#display(df.dtypes)

In [20]:
###################################################################
################ Train : Panne1 & Panne2           ################
################ Test  : Panne3                   #################
###################################################################

# Définir les périodes pour le train et le test
train_periods = [{'start': '2020-02-01 00:00:00', 'end': '2020-05-30 06:00:00'}]
test_periods  = [{'start': '2020-05-30 06:00:10', 'end': '2020-07-14 14:30:00'}]

# Affectations pour les périodes d'entraînement
start_train = pd.Timestamp(train_periods[0]['start'])
end_train = pd.Timestamp(train_periods[0]['end'])
train_indices = df_windowed[(df_windowed['timestamp'] >= start_train) & (df_windowed['timestamp'] <= end_train)].index.tolist()

# Affectations pour les périodes de test
start_test = pd.Timestamp(test_periods[0]['start'])
end_test = pd.Timestamp(test_periods[0]['end'])
test_indices = df_windowed[(df_windowed['timestamp'] >= start_test) & (df_windowed['timestamp'] <= end_test)].index.tolist()

# Créer les ensembles d'entraînement et de test
X_train = df_windowed.loc[train_indices].drop(columns=['timestamp', 'panne'])
y_train = df_windowed.loc[train_indices, 'panne']

X_test = df_windowed.loc[test_indices].drop(columns=['timestamp', 'panne'])
y_test = df_windowed.loc[test_indices, 'panne']

# Vérification des tailles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")


Taille de l'ensemble d'entraînement : 1030232
Taille de l'ensemble de test : 391860


In [21]:
# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)


Rapport de classification :


C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.98    372869
           1       0.85      0.01      0.03     18901
           2       0.00      0.00      0.00        90

    accuracy                           0.95    391860
   macro avg       0.60      0.34      0.33    391860
weighted avg       0.95      0.95      0.93    391860

Importance des features :
                  Feature  Importance
36       Reservoirs_lag_5    0.031776
1             DV_pressure    0.017396
13      DV_pressure_lag_1    0.017390
113    DV_pressure_lag_21    0.017241
98     DV_pressure_lag_18    0.017179
..                    ...         ...
450  Motor_current_lag_88    0.000043
205  Motor_current_lag_39    0.000041
170  Motor_current_lag_32    0.000040
215  Motor_current_lag_41    0.000034
8                     LPS    0.000000

[457 rows x 2 columns]


C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
###################################################################
################ Train : Panne1 & Panne2 & Panne3  ################
################ Test  : Panne4           #########################
###################################################################


train_periods = [{'start': '2020-02-01 00:00:00','end': '2020-06-07 14:30:00'}]

test_periods  = [{'start': '2020-06-07 14:30:10','end': '2020-09-01 03:59:50'}]

# Affectations pour les périodes d'entraînement
start_train = pd.Timestamp(train_periods[0]['start'])
end_train = pd.Timestamp(train_periods[0]['end'])
train_indices = df_windowed[(df_windowed['timestamp'] >= start_train) & (df_windowed['timestamp'] <= end_train)].index.tolist()

# Affectations pour les périodes de test
start_test = pd.Timestamp(test_periods[0]['start'])
end_test = pd.Timestamp(test_periods[0]['end'])
test_indices = df_windowed[(df_windowed['timestamp'] >= start_test) & (df_windowed['timestamp'] <= end_test)].index.tolist()

# Créer les ensembles d'entraînement et de test
X_train = df_windowed.loc[train_indices].drop(columns=['timestamp', 'panne'])
y_train = df_windowed.loc[train_indices, 'panne']

X_test = df_windowed.loc[test_indices].drop(columns=['timestamp', 'panne'])
y_test = df_windowed.loc[test_indices, 'panne']

# Vérification des tailles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")


Taille de l'ensemble d'entraînement : 1102412
Taille de l'ensemble de test : 739259


In [26]:
# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)


Rapport de classification :


C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    737548
           1       0.00      0.00      0.00      1621
           2       0.00      0.00      0.00        90

    accuracy                           1.00    739259
   macro avg       0.33      0.33      0.33    739259
weighted avg       1.00      1.00      1.00    739259

Importance des features :
                  Feature  Importance
178    DV_pressure_lag_34    0.027924
98     DV_pressure_lag_18    0.022352
238    DV_pressure_lag_46    0.022314
13      DV_pressure_lag_1    0.017224
1             DV_pressure    0.017211
..                    ...         ...
6              DV_eletric    0.000043
195  Motor_current_lag_37    0.000041
330  Motor_current_lag_64    0.000040
262            TP2_lag_51    0.000039
260  Motor_current_lag_50    0.000037

[457 rows x 2 columns]


In [27]:
##########################################################
################ Train : Panne2 & Panne3  ################
################ Test  : Panne4           ################
##########################################################

train_periods = [{'start': '2020-04-18 23:59:10','end': '2020-07-15 11:29:50'}]

test_periods = [{'start': '2020-07-15 11:30:00','end': '2020-09-01 03:59:50'}]

# Affectations pour les périodes d'entraînement
start_train = pd.Timestamp(train_periods[0]['start'])
end_train = pd.Timestamp(train_periods[0]['end'])
train_indices = df_windowed[(df_windowed['timestamp'] >= start_train) & (df_windowed['timestamp'] <= end_train)].index.tolist()

# Affectations pour les périodes de test
start_test = pd.Timestamp(test_periods[0]['start'])
end_test = pd.Timestamp(test_periods[0]['end'])
test_indices = df_windowed[(df_windowed['timestamp'] >= start_test) & (df_windowed['timestamp'] <= end_test)].index.tolist()

# Créer les ensembles d'entraînement et de test
X_train = df_windowed.loc[train_indices].drop(columns=['timestamp', 'panne'])
y_train = df_windowed.loc[train_indices, 'panne']

X_test = df_windowed.loc[test_indices].drop(columns=['timestamp', 'panne'])
y_test = df_windowed.loc[test_indices, 'panne']

# Vérification des tailles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")


Taille de l'ensemble d'entraînement : 755825
Taille de l'ensemble de test : 412020


In [28]:
# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)


Rapport de classification :


C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    410309
           1       0.00      0.00      0.00      1621
           2       0.00      0.00      0.00        90

    accuracy                           1.00    412020
   macro avg       0.33      0.33      0.33    412020
weighted avg       0.99      1.00      0.99    412020

Importance des features :
                    Feature  Importance
29    Oil_temperature_lag_4    0.047452
44    Oil_temperature_lag_7    0.034737
14    Oil_temperature_lag_1    0.027723
54    Oil_temperature_lag_9    0.021146
69   Oil_temperature_lag_12    0.020536
..                      ...         ...
10                Oil_level    0.000014
147              TP2_lag_28    0.000013
6                DV_eletric    0.000005
5                      COMP    0.000005
7                    Towers    0.000005

[457 rows x 2 columns]


C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
##########################################################
################ Train : Panne2 & Panne3 & Panne4   ######
################ Test  : Panne1           ################
##########################################################

train_periods = [{'start': '2020-04-18 23:59:10','end': '2020-09-01 03:59:50'}]

test_periods  = [{'start': '2020-02-01 00:00:00','end': '2020-04-18 23:59:00'}]

# Affectations pour les périodes d'entraînement
start_train = pd.Timestamp(train_periods[0]['start'])
end_train = pd.Timestamp(train_periods[0]['end'])
train_indices = df_windowed[(df_windowed['timestamp'] >= start_train) & (df_windowed['timestamp'] <= end_train)].index.tolist()

# Affectations pour les périodes de test
start_test = pd.Timestamp(test_periods[0]['start'])
end_test = pd.Timestamp(test_periods[0]['end'])
test_indices = df_windowed[(df_windowed['timestamp'] >= start_test) & (df_windowed['timestamp'] <= end_test)].index.tolist()

# Créer les ensembles d'entraînement et de test
X_train = df_windowed.loc[train_indices].drop(columns=['timestamp', 'panne'])
y_train = df_windowed.loc[train_indices, 'panne']

X_test = df_windowed.loc[test_indices].drop(columns=['timestamp', 'panne'])
y_test = df_windowed.loc[test_indices, 'panne']

# Vérification des tailles
print(f"Taille de l'ensemble d'entraînement : {len(X_train)}")
print(f"Taille de l'ensemble de test : {len(X_test)}")


Taille de l'ensemble d'entraînement : 1167845
Taille de l'ensemble de test : 673826


In [30]:
# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluation sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Importance des features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Importance des features :")
print(feature_importances)

Rapport de classification :


C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99    665101
           1       0.46      0.27      0.34      8635
           2       0.00      0.00      0.00        90

    accuracy                           0.99    673826
   macro avg       0.48      0.42      0.45    673826
weighted avg       0.98      0.99      0.98    673826

Importance des features :
                    Feature  Importance
29    Oil_temperature_lag_4    0.032554
179  Oil_temperature_lag_34    0.025332
44    Oil_temperature_lag_7    0.023352
14    Oil_temperature_lag_1    0.021492
54    Oil_temperature_lag_9    0.020326
..                      ...         ...
307              TP2_lag_60    0.000021
6                DV_eletric    0.000016
352              TP2_lag_69    0.000016
5                      COMP    0.000010
9           Pressure_switch    0.000008

[457 rows x 2 columns]
